Import the required libraries

In [ ]:
! pip install transformers openai numpy requests

The below function returns the contextualized word embeddings from gpt2

In [ ]:
from transformers import GPT2Tokenizer
import numpy as np
import json
import os, openai, requests
#OPENAI_API_KEY = os.environ.get("sk-ckCerHbXrCSeaf61c9ELvsy2EcUEz0w2SxBCsIeD")
OPENAI_API_KEY = 'sk-ckCerHbXrCSeaf61c9ELvsy2EcUEz0w2SxBCsIeD'
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
def get_embeddings(tokens):
   tokens = ', '.join(tokens)
   tokens = "[[" +  tokens + "]]"
   headers = {
       "Authorization": f"Bearer {OPENAI_API_KEY}",
       "Content-Type": "application/json",
   }
   data = (
       '{"prompt":' + f"{tokens}" + '}'
   )
   response = requests.post(
       "https://api.openai.com/v1/engines/ada-search-index-v1/embeddings",
       headers=headers,
       data=data.encode("utf-8"),
   )
   content = json.loads(response.text)
   return content   
   #return content["data"][0]["activations"]


The encode_sent function returns the word_encoding after pre-processing

In [ ]:
def encode_sent(sent, type_data):
   tokens = tokenizer(sent)['input_ids']

    # if condition to check if it is a query or a document
   if type_data == 'query':
    print('This is a query')
    tokens = tokenizer("[")['input_ids'] + tokens + tokenizer("]")['input_ids'] # query embeddings
   else:
    tokens = tokenizer("{")['input_ids'] + tokens + tokenizer("}")['input_ids'] # doc embeddings
   if len(tokens) > 2048:
      tokens = tokens[-2048:]
   tokens = [str(t) for t in tokens]
   return tokens


In [ ]:
# call the function to get the results
def embed(txt, type_data):
   tokens = encode_sent(txt,type_data)
   return get_embeddings(tokens)

In [ ]:
# to calculate the cosine similarity
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

# Build a simple semantic search

Upload the file to extract the word embeddings and repalce with the filename - trial2.json

In [ ]:
document_dictionary = {}

# Python program to read
# json file
import json

# Enter the json file name
f = open('all_data.json',encoding="utf8")

# returns JSON object as a dictionary
data = json.load(f)

counter = 0
# Iterating through the json list
for i in data['lexata']:
    st = 'doc_' + str(counter)
    document_dictionary[st] = i['metadata'], i['text']
    counter = counter + 1

# Closing file
f.close()

In [ ]:
len(document_dictionary.keys())

1862

In [ ]:
 # create an empty list which will store the word embeddings and will be indexed
word_embed_dictionary = {}

counter = 0

for i in document_dictionary.keys():
  st = 'doc_'+ str(counter)
  results = embed(document_dictionary.get(i)[1], type_data='doc')
  final_results = results["data"][0]["activations"]
  word_embed_dictionary[st] = final_results
  print('doc embedding completed:',counter)
  counter = counter + 1

doc embedding completed: 0
doc embedding completed: 1
doc embedding completed: 2
doc embedding completed: 3
doc embedding completed: 4
doc embedding completed: 5
doc embedding completed: 6
doc embedding completed: 7
doc embedding completed: 8
doc embedding completed: 9
doc embedding completed: 10
doc embedding completed: 11
doc embedding completed: 12
doc embedding completed: 13
doc embedding completed: 14
doc embedding completed: 15
doc embedding completed: 16
doc embedding completed: 17
doc embedding completed: 18
doc embedding completed: 19
doc embedding completed: 20
doc embedding completed: 21
doc embedding completed: 22
doc embedding completed: 23
doc embedding completed: 24
doc embedding completed: 25
doc embedding completed: 26
doc embedding completed: 27
doc embedding completed: 28
doc embedding completed: 29
doc embedding completed: 30
doc embedding completed: 31
doc embedding completed: 32
doc embedding completed: 33
doc embedding completed: 34
doc embedding completed: 35
do

In [ ]:
# create an empty list to extract the word embeddings
word_embed_values = []

for i in word_embed_dictionary.keys():
  word_embed_values.append(word_embed_dictionary.get(i))


To extract the embed values of 1800 documents to csv

In [ ]:
! pip uninstall pandas
!pip install pandas==1.1.5

Found existing installation: pandas 1.3.3
Uninstalling pandas-1.3.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/pandas-1.3.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/pandas/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/pandas/__init__.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/__init__.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/config.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/dates.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/display.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/localization.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/__init__.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/indexing.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/interval.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/missing.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/pr

In [ ]:
# to extract the word embedding and the corresponding post_id and doc_id to dictionary

import pandas as pd

assign_doc_id = []
post_id = []

for index_i, i in enumerate(word_embed_values):
  st = 'doc_' + str(index_i)
  assign_doc_id.append(st)

for i in document_dictionary.keys():
  post_id.append(document_dictionary.get(i)[0])

data = pd.DataFrame(
    {'document_id': assign_doc_id,
     'post_id': post_id,
     'word embedding': word_embed_values
    })

data.to_csv('word_embedding.csv',index=False)

In [ ]:
def calculate_cosine_similarity(vec1, vec2):
  return cosine_similarity([vec1],[vec2])  

# query the documents 

In [ ]:
# type the query between the quotes '' 
txt = 'insider report deadline how many days'



results = embed(txt, type_data='query')
response = results["data"][0]["activations"]

# create an empty dictionary
semantic_results = {}

for i in range(len(word_embed_values)):
  st = 'doc_' + str(i)
  res = calculate_cosine_similarity(response, word_embed_values[i])
  semantic_results[st] = res

  # sorts the document and displays the results
sort_orders = sorted(semantic_results.items(), key=lambda x: x[1], reverse=True)
for i in sort_orders:
  print('doc id : ', i[0])
  print('semantic_score : ', i[1])
  print(document_dictionary.get(i[0]))
  print('=============================')
  

In [ ]:
semantic_results

Posts recommendation

In [ ]:
post_1 = word_embed_values[10]

#for i in word_embed_values:
calculate_cosine_similarity(word_embed_values[0],post_1)


array([[0.69532913]])

In [ ]:
# list to store the main document id
main_doc_id = []
main_post_id = []
cosine_similarity_li = []
similar_doc_id = []
similar_post_id = []


for index_i, i in enumerate (word_embed_values):
  post_recommend = {}
  
  main_st = 'doc_' + str(index_i)
  for index_j, j in enumerate (word_embed_values):
    if(index_i != index_j):

      # doc_id
      similar_st = 'doc_' + str(index_j)
      
      # find the cosine similarity of the documents
      res = calculate_cosine_similarity(i,j)

      # add the values to dictionary
      post_recommend[similar_st] = res

      # sort the values
      sort_doc_recommend = sorted(post_recommend.items(), key=lambda x: x[1], reverse=True)

  # after the for loop j is completed
  # get the top 5 similar posts
  for k in sort_doc_recommend[0:5]:

    # add the main doc id to the list
    main_doc_id.append(main_st)

    # add the main post id to the list
    main_post_id.append(document_dictionary.get(main_st)[0])

    # add the cosine similarity to the list
    cosine_similarity_li.append(k[1])

    # add the similar doc_id to the list
    similar_doc_id.append(k[0])

    # add the similar post_id to the list
    similar_post_id.append(document_dictionary.get(k[0])[0])

  print('Top 5 similar documents for doc : ',index_i,' completed.')


post_recommend_data = pd.DataFrame(
    {'Main_doc_id': main_doc_id,
     'Main_post_id': main_post_id,
     'Score': cosine_similarity_li,
     'Similar_doc_id': similar_doc_id,
     'Similar_post_id': similar_post_id,
    })
    
post_recommend_data.head(10)

# extract the data to csv
post_recommend_data.to_csv('post_recommendation.csv')

Top 5 similar documents for doc :  0  completed.
Top 5 similar documents for doc :  1  completed.
Top 5 similar documents for doc :  2  completed.
Top 5 similar documents for doc :  3  completed.
Top 5 similar documents for doc :  4  completed.
Top 5 similar documents for doc :  5  completed.
Top 5 similar documents for doc :  6  completed.
Top 5 similar documents for doc :  7  completed.
Top 5 similar documents for doc :  8  completed.
Top 5 similar documents for doc :  9  completed.
Top 5 similar documents for doc :  10  completed.
Top 5 similar documents for doc :  11  completed.
Top 5 similar documents for doc :  12  completed.
Top 5 similar documents for doc :  13  completed.
Top 5 similar documents for doc :  14  completed.
Top 5 similar documents for doc :  15  completed.
Top 5 similar documents for doc :  16  completed.
Top 5 similar documents for doc :  17  completed.
Top 5 similar documents for doc :  18  completed.
Top 5 similar documents for doc :  19  completed.
Top 5 simi